# 2.4 Statistical Decision Theory

#### Loss function

The most common is *squared error loss*: $L(Y, f(X)) = (Y - f(X))^2$

#### Expected prediction Error
(2.9, 2.10) This leads us to a criterion for choosing $f$, 
 
$$EPE(f) = E(Y - f(X))^2 = \int{[y - f(x)]^2}Pr(dx, dy)$$

(2.11) By conditioning on X, we can write EPE as: 

$$EPE(f) = E_X E_{Y|X} ([Y - f(X)]^2|X) $$

Proof:

\begin{equation}
 \int{[y - f(x)]^2}Pr(dx, dy) \\
 = \int{[y - f(x)]^2}p(x, y)dxdy \\ 
 = \int{[y - f(x)]^2}p(x)p(y | x)dxdy \\
 = \int_x { \left( \int_y {[y - f(x)]^2p(y | x)dy} \right)p(x)dx } \\
 = E_X E_{Y|X} ([Y - f(X)]^2|X)
\end{equation}


(2.12) It suffices to minimize EPE pointwise: $$f(x) = argmin_c E_{Y|X} ( [Y - c]^2 | X = x)$$

(2.13) The solution is : $$f(x)=E(Y | X = x)$$